# CrewAI Example: NBA Data Analysis

This notebook demonstrates a complete NBA data analysis application using CrewAI's multi-agent framework.

**All functions are imported from `crewai_utils.py`** - keeping the notebook clean and focused on demonstrating functionality.


## Setup

Set your OpenAI API key and import functions from `crewai_utils.py`.


In [ ]:
import os

# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'  # Replace with your actual key

# Import all functions from crewai_utils
from crewai_utils import (
    # Configuration
    NBA_DATA_PATH, get_llm,
    # Vector DB
    NBAVectorDB, get_vector_db,
    # Tools
    get_agent_tools,
    # Agents
    create_engineer_agent, create_analyst_agent, create_storyteller_agent,
    # Tasks
    create_data_engineering_task, create_data_analysis_task, 
    create_custom_analysis_task, create_storyteller_task,
    # Crews
    create_crew, create_flow_crew, create_analyst_only_crew,
    # App functions
    process_file_and_analyze, process_question_only,
    # Main
    main
)

print("All imports successful!")
print(f"Default data path: {NBA_DATA_PATH}")


## Example 1: Basic Crew Execution

Run a basic analysis with Engineer and Analyst agents using functions from `crewai_utils.py`.


In [ ]:
# Create and run a basic crew (using function from crewai_utils.py)
crew = create_crew()

print("Starting crew execution...")
result = crew.kickoff()

print("\n" + "="*70)
print("RESULTS")
print("="*70)
print(result)


## Example 2: Custom Query Analysis

Ask a specific question and get analysis results with all three agents (Engineer, Analyst, Storyteller).


In [ ]:
# Define your query
user_query = "Who are the top 5 three-point shooters?"

# Create crew with custom query (using function from crewai_utils.py)
csv_path = NBA_DATA_PATH
crew = create_flow_crew(user_query, csv_path)

print(f"Analyzing: {user_query}")
print("Starting crew execution...\n")

result = crew.kickoff()

# Display results
print("\n" + "="*70)
print("ANALYSIS RESULTS")
print("="*70)

if hasattr(result, 'tasks_output') and result.tasks_output:
    print("\nEngineer Output:")
    print(result.tasks_output[0] if len(result.tasks_output) > 0 else "N/A")
    
    print("\n" + "-"*70)
    print("\nAnalyst Output:")
    print(result.tasks_output[1] if len(result.tasks_output) > 1 else "N/A")
    
    print("\n" + "-"*70)
    print("\nStoryteller Output:")
    print(result.tasks_output[2] if len(result.tasks_output) > 2 else "N/A")
else:
    print(result)


## Example 3: Analyst Only (Quick Questions)

For quick questions, use only the Analyst agent (faster, no Engineer/Storyteller overhead).


In [ ]:
# Quick question - Analyst only (using function from crewai_utils.py)
query = "What team has the best win rate?"

crew = create_analyst_only_crew(query, NBA_DATA_PATH)

print(f"Question: {query}")
print("Running analysis...\n")

result = crew.kickoff()

print("="*70)
print("ANSWER")
print("="*70)
print(result.tasks_output[0] if hasattr(result, 'tasks_output') and result.tasks_output else result)


## Example 4: Using Individual Components

You can also use individual agents and tasks separately (all from `crewai_utils.py`).


In [ ]:
# Create agents individually (using functions from crewai_utils.py)
engineer = create_engineer_agent(NBA_DATA_PATH)
analyst = create_analyst_agent(NBA_DATA_PATH)
storyteller = create_storyteller_agent()

print("Agents created:")
print(f"  - Engineer: {engineer.role}")
print(f"  - Analyst: {analyst.role}")
print(f"  - Storyteller: {storyteller.role}")

# Create tasks (using functions from crewai_utils.py)
eng_task = create_data_engineering_task(engineer, NBA_DATA_PATH)
analysis_task = create_custom_analysis_task(
    analyst, 
    "Find the top 10 players by total points",
    None,
    NBA_DATA_PATH
)
story_task = create_storyteller_task(storyteller, analysis_task)

# Create custom crew
from crewai import Crew, Process

custom_crew = Crew(
    agents=[engineer, analyst, storyteller],
    tasks=[eng_task, analysis_task, story_task],
    process=Process.sequential,
    verbose=True
)

print("\nRunning custom crew...")
result = custom_crew.kickoff()

print("\n" + "="*70)
print("CUSTOM CREW RESULTS")
print("="*70)
print(result)


## Example 5: Using Tools Directly

You can also use the tools directly without agents (all from `crewai_utils.py`).


In [ ]:
# Get tools (using function from crewai_utils.py)
tools = get_agent_tools(NBA_DATA_PATH)

# Use tools directly
print("Getting data summary...")
summary_tool = [t for t in tools if t.name == "get_nba_data_summary"][0]
summary = summary_tool.run()
print(summary)

print("\n" + "="*70)

# Use analyze tool directly
print("\nAnalyzing top 5 three-point shooters...")
analyze_tool = [t for t in tools if t.name == "analyze_nba_data"][0]
analysis = analyze_tool.run("df.groupby('Player')['3P'].sum().sort_values(ascending=False).head(5)")
print(analysis)


## Example 6: Vector Database Usage

Use semantic search for finding similar records (using `get_vector_db` from `crewai_utils.py`).


In [ ]:
# Get vector database instance (using function from crewai_utils.py)
vector_db = get_vector_db(NBA_DATA_PATH)

# Perform semantic search
query = "high scoring games with many three pointers"
results = vector_db.search(query, n_results=5)

print(f"Semantic search for: '{query}'")
print(f"Found {len(results)} results\n")

for i, result in enumerate(results, 1):
    print(f"Result {i} (Similarity: {result['similarity']:.3f}):")
    print(f"  {result['document']}")
    print(f"  Player: {result['metadata'].get('player', 'N/A')}")
    print(f"  Points: {result['metadata'].get('points', 'N/A')}")
    print()


## Example 7: Multiple Queries

Run multiple analyses in sequence (all using functions from `crewai_utils.py`).


In [ ]:
# List of queries to analyze
queries = [
    "Who are the top 5 players by assists?",
    "Which team has the highest average points per game?",
    "Find players with the best field goal percentage (minimum 10 attempts)"
]

results_dict = {}

for i, query in enumerate(queries, 1):
    print(f"\n{'='*70}")
    print(f"Query {i}/{len(queries)}: {query}")
    print('='*70)
    
    # Use function from crewai_utils.py
    crew = create_analyst_only_crew(query, NBA_DATA_PATH)
    result = crew.kickoff()
    
    results_dict[query] = result.tasks_output[0] if hasattr(result, 'tasks_output') and result.tasks_output else str(result)
    
    print(f"\nCompleted query {i}")
    print("-"*70)

# Display all results
print("\n\n" + "="*70)
print("ALL RESULTS SUMMARY")
print("="*70)

for query, answer in results_dict.items():
    print(f"\n{query}")
    print(f"{answer[:200]}..." if len(answer) > 200 else f"{answer}")
    print()


## Summary

This notebook demonstrated how to use functions from `crewai_utils.py` to:
- Create and run basic crews
- Perform custom query analysis
- Use analyst-only for quick questions
- Use individual components (agents, tasks)
- Use tools directly
- Perform semantic search with vector database
- Run multiple queries in sequence

**Key Point**: All complex logic is in `crewai_utils.py`. This notebook demonstrates usage by calling those functions, keeping the code clean and maintainable.
